In [3]:
!pip install sentence_transformers
!pip install numpy
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 60.6 MB/s eta 0:00:00


In [4]:
# Intent classification using embeddings + nearest neighbor similarity
# Author: ChatGPT (GPT-5)
# Requirements: pip install sentence-transformers faiss-cpu numpy

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# -----------------------------
# Step 1: Define your intents and example utterances
# -----------------------------
intent_examples = {
    "book_flight": [
        "Book me a flight to New York",
        "I need to reserve a plane ticket",
        "Find flights from Delhi to Mumbai",
        "Schedule a trip by air"
    ],
    "weather_query": [
        "What's the weather like today?",
        "Is it going to rain tomorrow?",
        "Tell me the forecast for Chennai",
        "Temperature in Bangalore right now"
    ],
    "greeting": [
        "Hey there!",
        "Good morning",
        "Hello",
        "Hi, how are you?"
    ],
    "goodbye": [
        "Bye",
        "See you later",
        "Talk to you soon",
        "Good night"
    ]
}

# -----------------------------
# Step 2: Load embedding model
# -----------------------------
model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast and accurate

# -----------------------------
# Step 3: Create embeddings for all examples
# -----------------------------
sentences = []
labels = []
for intent, examples in intent_examples.items():
    sentences.extend(examples)
    labels.extend([intent] * len(examples))

embeddings = model.encode(sentences, normalize_embeddings=True)

# -----------------------------
# Step 4: Build FAISS index
# -----------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # inner product = cosine similarity (since normalized)
index.add(embeddings)

print(f"Indexed {len(sentences)} utterances across {len(intent_examples)} intents.")

# -----------------------------
# Step 5: Define a prediction function
# -----------------------------
def predict_intent(query, threshold=0.6):
    query_emb = model.encode([query], normalize_embeddings=True)
    D, I = index.search(query_emb, k=3)  # get top 3 matches
    top_score = D[0][0]
    top_label = labels[I[0][0]]

    if top_score < threshold:
        return {"intent": "none", "confidence": float(top_score), "match_examples": []}

    matches = [(labels[i], sentences[i], float(D[0][j])) for j, i in enumerate(I[0])]
    return {"intent": top_label, "confidence": float(top_score), "match_examples": matches}

# -----------------------------
# Step 6: Try with test queries
# -----------------------------
test_queries = [
    "Can you check flights to Singapore?",
    "How hot is it today?",
    "See you!",
    "Yo!"
]

for q in test_queries:
    result = predict_intent(q)
    print(f"\nQuery: {q}")
    print(f"→ Predicted Intent: {result['intent']} (confidence={result['confidence']:.2f})")
    for match in result["match_examples"]:
        print(f"   matched: {match[1]} (intent={match[0]}, score={match[2]:.2f})")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Indexed 16 utterances across 4 intents.

Query: Can you check flights to Singapore?
→ Predicted Intent: none (confidence=0.38)

Query: How hot is it today?
→ Predicted Intent: weather_query (confidence=0.77)
   matched: What's the weather like today? (intent=weather_query, score=0.77)
   matched: Temperature in Bangalore right now (intent=weather_query, score=0.46)
   matched: Is it going to rain tomorrow? (intent=weather_query, score=0.40)

Query: See you!
→ Predicted Intent: none (confidence=0.59)

Query: Yo!
→ Predicted Intent: none (confidence=0.40)


In [8]:
query_emb = model.encode(["How hot is it today?"], normalize_embeddings=True)
D, I = index.search(query_emb, k=3)  # get top 3 matches

To generate OpenAI embeddings, you typically use their official Python library and provide your API key. However, if you have an OAuth setup with a client ID, secret, and token endpoint, you would first need to obtain an access token using these credentials and then use that token to authenticate with the OpenAI API.

Here's a general approach using the `requests` library to get the access token and then demonstrating how to use the OpenAI library with a hypothetical token (you'll need to replace placeholders with your actual values and handle token expiration/refreshing).

**Note:** The standard OpenAI library primarily uses API keys. Using OAuth directly with the OpenAI library might require custom handling of the authorization header. This example shows the token acquisition and a conceptual way to use it.

In [ ]:
import httpx
import openai
import os
import time

# Replace with your actual credentials and endpoint
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
token_url = "YOUR_OAUTH_TOKEN_ENDPOINT" # e.g., "https://auth.example.com/oauth/token"

# Store token information globally or in a class instance
access_token_info = {
    "access_token": None,
    "expires_at": 0 # Unix timestamp
}

# Function to get or refresh the access token
def get_access_token(client: httpx.Client):
    global access_token_info

    # Check if the current token is still valid
    # Add a small buffer (e.g., 60 seconds) to avoid using expired tokens
    if access_token_info["access_token"] and access_token_info["expires_at"] > time.time() + 60:
        print("Reusing existing access token.")
        return access_token_info["access_token"]

    print("Obtaining new access token...")
    try:
        token_data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret
        }
        response = client.post(token_url, data=token_data)
        response.raise_for_status() # Raise an exception for bad status codes
        token_info = response.json()
        access_token = token_info.get("access_token")
        expires_in = token_info.get("expires_in") # Time in seconds until expiration

        if not access_token:
            raise ValueError("Access token not found in the response.")
        if expires_in is None:
             # Assume a default expiration or handle appropriately if not provided
             print("Warning: 'expires_in' not provided in token response. Token reuse may be unreliable.")
             expires_at = time.time() + 3600 # Default to 1 hour if not specified
        else:
             expires_at = time.time() + expires_in


        access_token_info["access_token"] = access_token
        access_token_info["expires_at"] = expires_at

        print("Successfully obtained and stored access token.")
        return access_token

    except httpx.RequestError as e:
        print(f"Error obtaining access token: {e}")
        return None
    except ValueError as e:
        print(f"Error processing token response: {e}")
        return None

# 2. Use the access token with the OpenAI API
# A more robust approach for OAuth is to use httpx with custom headers
# for API calls.

# Example of getting embeddings using the httpx library and the token:
# (You would need to find the specific OpenAI embeddings endpoint URL)
# embeddings_url = "https://api.openai.com/v1/embeddings" # Example endpoint

# Create an httpx client for making requests
with httpx.Client() as client:
    access_token = get_access_token(client)

    if access_token:
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }
        data = {
            "input": "Your text string to embed",
            "model": "text-embedding-ada-002" # Specify the model
        }
        # try:
        #     embeddings_response = client.post(embeddings_url, json=data, headers=headers)
        #     embeddings_response.raise_for_status()
        #     embeddings_data = embeddings_response.json()
        #     print("\nEmbeddings generated successfully:")
        #     # Process embeddings_data
        #     # print(embeddings_data)
        # except httpx.RequestError as e:
        #     print(f"Error generating embeddings: {e}")
    else:
        print("\nCould not proceed with generating embeddings as access token was not obtained.")